# Control Tutorial_9
_Reference : The Control Handbook, Control System Fundamentals, Edited by William S.Levine from p5-83 to p5-85_, https://www.youtube.com/watch?v=E45v2dD3IQU

# Finite State Machines
---
Finite State Machine (FSM) is strictly logical model where the system remains in each state for an unspecified amount of time and the transitions between states are triggered by events that occur instantaneously. It is often used to for artificial intelligences.

# Simulation
## Example (RobotMaid)
<img src="figures/FSM_robotmaid.png" hight="150">

We consider the robot maid which can work on three kind of action.  
The system has three States, "CleanDishes", "Vacuum", and "Sleep", and three transitions, "toSleep", and "toCleanDishes", "toVacuum".  
In this notebook, we don't use any specific libraries for FSM. Here is the code for this system. 
The `the random.randint` method is used for making timer for the trigger of the Transitions.

In [1]:
from random import randint
from time import clock

Declare the super classes for Transition and State class.

In [2]:
class Transition(object):
    def __init__(self,toState):
        self.toState = toState
        
    def Execute(self):
        print("=======Transitioning=======")

In [3]:
class State(object):
    def __init__(self,FSM):
        self.FSM = FSM
        self.timer = 0
        self.startTime = 0
        
    def Enter(self):
        # Set the timer for the trigger
        self.timer = randint(0,5)
        self.startTime = int(clock())
        
    def Execute(self):
        pass
    def Exit(self):
        pass

Declare the State that the system has. Each State has three functions, `Enter`, `Execute`, and `Exit`.

In [4]:
class CleanDishes(State):
    def __init__(self,FSM):
        super().__init__(FSM)
    
    def Enter(self):
        print("Preparing to clean dishes.")
        super().Enter()
        
    def Execute(self):
        print("Cleaning dishes")
        if self.startTime+self.timer<=clock(): #Trigger
            if randint(1,3)%2:
                self.FSM.ToTransition("toVacuum")
            else:
                self.FSM.ToTransition("toSleep")
    
    def Exit(self):
        print("Finished cleaning dishes.")

In [5]:
class Vacuum(State):
    def __init__(self,FSM):
        super().__init__(FSM)
    
    def Enter(self):
        print("Preparing to Vacuum.")
        super().Enter()
        
    def Execute(self):
        print("Vacuuming")
        if self.startTime+self.timer<=clock(): #Trigger
            if randint(1,3)%2:
                self.FSM.ToTransition("toSleep")
            else:
                self.FSM.ToTransition("toCleanDishes")
    
    def Exit(self):
        print("Finished Vacuuming.")

In [6]:
class Sleep(State):
    def __init__(self,FSM):
        super().__init__(FSM)
        self.sleepAmount = 0
        self.startTime = 0
    
    def Enter(self):
        print("Starting to sleep.")
        super().Enter()
        
    def Execute(self):
        print("Sleeping")
        if self.startTime+self.timer<=clock(): #Trigger
            if randint(1,3)%2:
                self.FSM.ToTransition("toVacuum")
            else:
                self.FSM.ToTransition("toCleanDishes")
    
    def Exit(self):
        print("Waking up from Sleep.")

In [7]:
class FSM(object):
    def __init__(self,charactor):
        self.char = charactor
        self.states = {}
        self.transitions = {}
        self.curState = None
        self.prevState = None
        self.trans = None
    
    def AddTransition(self,transName,transition):
        self.transitions[transName] = transition
    
    def AddState(self,stateName,state):
        self.states[stateName] = state
        
    def SetState(self,stateName):
        self.prevState = self.curState
        self.curState = self.states[stateName]
    
    def ToTransition(self,toTrans):
        self.trans = self.transitions[toTrans]
        
    def Execute(self):
        if self.trans:
            self.curState.Exit()
            self.trans.Execute()
            self.SetState(self.trans.toState)
            self.curState.Enter()
            self.trans = None
        self.curState.Execute()

In [8]:
# Implementation
Char = type("Char",(object,),{})

class RobotMaid(Char):
    def __init__(self):
        self.FSM = FSM(self)
        
        # States
        self.FSM.AddState("Sleep",Sleep(self.FSM))
        self.FSM.AddState("CleanDishes",CleanDishes(self.FSM))
        self.FSM.AddState("Vacuum",Vacuum(self.FSM))
        
        # Transitions
        self.FSM.AddTransition("toSleep",Transition("Sleep"))
        self.FSM.AddTransition("toVacuum",Transition("Vacuum"))
        self.FSM.AddTransition("toCleanDishes",Transition("CleanDishes"))
        
        # Initial State
        self.FSM.SetState("Sleep")
        
    def Execute(self):
        self.FSM.Execute()

We simulate the 20 iteration for every second.

In [9]:
r = RobotMaid()
for i in range(20):
    startTime = clock()
    timeInterval = 1
    while (startTime+timeInterval) > clock():
        pass
    r.Execute()

Sleeping
Waking up from Sleep.
=======Transitioning=======
Preparing to Vacuum.
Vacuuming
Vacuuming
Vacuuming
Finished Vacuuming.
=======Transitioning=======
Preparing to clean dishes.
Cleaning dishes
Cleaning dishes
Cleaning dishes
Cleaning dishes
Finished cleaning dishes.
=======Transitioning=======
Preparing to Vacuum.
Vacuuming
Vacuuming
Vacuuming
Vacuuming
Finished Vacuuming.
=======Transitioning=======
Starting to sleep.
Sleeping
Sleeping
Waking up from Sleep.
=======Transitioning=======
Preparing to Vacuum.
Vacuuming
Vacuuming
Finished Vacuuming.
=======Transitioning=======
Preparing to clean dishes.
Cleaning dishes
Cleaning dishes
Cleaning dishes
Cleaning dishes
